In [86]:
import collections
import itertools
import datetime

from nltk import Tree
from nltk.draw.util import CanvasFrame
from nltk.draw import TreeWidget
import nltk 

%matplotlib inline

grammar = nltk.data.load("grammars/large_grammars/atis.cfg") #load the grammar
#sentences = nltk.data.load("grammars/large_grammars/atis_sentences.txt") #load the raw sentences
#test_sentences = nltk.parse.util.extract_test_sentences(sentences) #extract the test sentences

data = open("atis/atis-grammar-cnf.cfg",'r').read() #read the file
grammar = nltk.grammar.CFG.fromstring(data)
test_sentences = nltk.data.load("atis/atis-test-sentences.txt") #load the test sentences
test_sentences = nltk.parse.util.extract_test_sentences(test_sentences) #extract the test sentences
grammar.productions(lhs = nltk.grammar.Nonterminal("SIGMA")) #convert the string to Nonterminal and get production rule for that Nonterminal
start_symbol = grammar.start()

def cky_recognizer(sentence):
    """
    This function searches the language of the CFG for a given sentence.
    Parameters:
    Sentence (list): this is the list of a string.
    Return:
    False: if the sentence is not in the grammar.
    Dictionary(defaultdict with list), Matrix(list of list of sets) if the sentence is in the grammar.
    """
    dictionary = collections.defaultdict(list)
    n = len(sentence)
    matrix = [[set() for j in range(n+1) ] for i in range(n+1)] #define matrix as set of nonterminal symbols

    for i in range(0,n): #iterate over all positions in string
        for rule in grammar.productions(rhs = sentence[i]): #look at rhs terminals of production rules
            matrix[i][i+1].add(rule.lhs())
            dictionary[(rule.lhs(),i,i+1)].append(sentence[i]) 
    for i in range(2, n+1): #iterate over substrings of increasing length
        for j in range(0,n-i+1): #iterate over start position of respective substring
            for k in range(1,i):
                B = matrix[j][j+k] #left part of substring j-j+k
                C = matrix[j+k][j+i] #right part of substrink j+k,j+i
                for each_b in B: #iterative over all nonterminal symbols found for left substring
                    for each_production in grammar.productions(rhs = each_b):
                        #iterate over all nonterminal symbols found for right substring
                        if each_production.rhs()[1] in C:
                            matrix[j][j+i].add(each_production.lhs())
                            dictionary[(each_production.lhs(),j,j+i)].append((each_production.rhs()[0],each_production.rhs()[1],j,j+k,j+i)) #append all found nonterminal symbols to list
    if start_symbol in matrix[0][n]: #checking if given sentence has start symbol or not
        return dictionary, matrix
    else: 
        print("The given sentence is not in the language of CFG\n\n")
        return None, None
    
def tree_parser(parent_node,i,j, backpointer):
    """
    This function extracts the set of all parse trees from the backpointers in the chart.
    Parameters: 
    parent_node(nltk.grammar.Nonterminal)
    i(int): row value for parent in matrix
    j(int): column value for parent in matrix
    backpointer(dict): from cky recognizer
    Return:
    List of trees
    """
    trees = []
    parent_values = backpointer[(parent_node,i,j)]
    
    for parent_value in parent_values: 
        if isinstance(parent_value,str): #checking type, whether the parent has terminal or nonterminal
            return [(parent_node,parent_value)]
        else:
            leftchild = tree_parser(parent_value[0],parent_value[2],parent_value[3],backpointer) #getting tree for leftchild
            rightchild = tree_parser(parent_value[1],parent_value[3],parent_value[4],backpointer) #getting tree for rightchild
            for left,right in itertools.product(leftchild,rightchild): #getting cartesian product multiple trees from the left and the right chlid
                trees.append((parent_node,left,right)) #appending trees to list
    return trees

### Part 1 - Recognizer: checks for sentence in test sentences, if the sentence is in grammar ###

In [75]:
time_recognizer = datetime.datetime.now() 

for i,sentence in enumerate(test_sentences): 
    print('sentence '+ str(i) + ': ' + ' '.join(sentence[0]))
    backpointer, chart = cky_recognizer(sentence[0])
    if backpointer and chart:
        print("The given CKY_Parser sentence is in the language of CFG \n\n")

time_n = datetime.datetime.now()
print("The total time taken was:", (time_n-time_recognizer).total_seconds(), "seconds") #measure total time to test, if all sentences are in the grammar

sentence 0: prices .
The given CKY_Parser sentence is in the language of CFG 


sentence 1: show availability .
The given CKY_Parser sentence is in the language of CFG 


sentence 2: show the flights .
The given CKY_Parser sentence is in the language of CFG 


sentence 3: milwaukee to detroit .
The given CKY_Parser sentence is in the language of CFG 


sentence 4: indianapolis to seattle .
The given CKY_Parser sentence is in the language of CFG 


sentence 5: list round trips .
The given CKY_Parser sentence is in the language of CFG 


sentence 6: list saturday flights .
The given CKY_Parser sentence is in the language of CFG 


sentence 7: what aircraft is this .
The given sentence is not in the language of CFG


sentence 8: list these economy fares .
The given sentence is not in the language of CFG


sentence 9: list these city destinations .
The given sentence is not in the language of CFG


sentence 10: what is the fare .
The given CKY_Parser sentence is in the language of CFG 


s

The given CKY_Parser sentence is in the language of CFG 


sentence 72: how long is the stop in dallas on flight d l nine thirty one .
The given CKY_Parser sentence is in the language of CFG 


sentence 73: please show me all first class flights from pittsburgh to newark on monday morning .
The given CKY_Parser sentence is in the language of CFG 


sentence 74: please show me all first class flights from indianapolis to memphis leaving monday morning .
The given CKY_Parser sentence is in the language of CFG 


sentence 75: please show me all flights from ontario to salt lake city leaving monday morning .
The given CKY_Parser sentence is in the language of CFG 


sentence 76: i need an american airlines flight number from chicago to philadelphia departing six p.m .
The given CKY_Parser sentence is in the language of CFG 


sentence 77: for american airlines i need round trip airfare from new york to san diego .
The given CKY_Parser sentence is in the language of CFG 


sentence 78: how 

In [77]:
ungrammaticals = ["the plane crashed in the middle of the ocean",
"the company produces mechanical parts for airplane engines",
"the airport is not well equipped to handle aircraft of such size",
"the accident had a lasting influence on the industry",
"the crew were all russian while the passengers were of various nationalities"]

for i,ungrammatical in enumerate(ungrammaticals): #iterate list, print ungrammatical sentences
    sentence = ungrammatical.split() #split each sentence
    print('sentence '+ str(i) + ': ' + ' '.join(sentence)+ "\n")
    backpointer, chart = cky_recognizer(sentence) #call CKY recognizer
    if backpointer and chart:
        print("The given CKY_Parser CKY_Parser sentence is in the language of CFG \n\n") #print if ungrammatical sentences are in grammar or not

sentence 0: the plane crashed in the middle of the ocean

The given sentence is not in the language of CFG


sentence 1: the company produces mechanical parts for airplane engines

The given sentence is not in the language of CFG


sentence 2: the airport is not well equipped to handle aircraft of such size

The given sentence is not in the language of CFG


sentence 3: the accident had a lasting influence on the industry

The given sentence is not in the language of CFG


sentence 4: the crew were all russian while the passengers were of various nationalities

The given sentence is not in the language of CFG




Feeding the recognizer ungrammatical sentences (ungrammaticals.txt), i.e. sentences which are not captured by the language of the CFG, one can observe that the recognizer rejects them.

### Part 2 - Parser: extracts the set of all parse trees from the backpointers in the chart###

In [81]:
time_parser = datetime.datetime.now()

output_file = open('trees_number.txt',"w") #open and write file
for sentence in test_sentences: 
    n = len(sentence)
    backpointer, chart = cky_recognizer(sentence[0]) #recognizer
    if backpointer and chart:
        parses = tree_parser(parent_node=start_symbol,i= 0,j=len(sentence[0]),backpointer=backpointer) 
        output_file.write((" ").join(sentence[0])+'\t'+ str(len(parses))+'\n') #writing file sentences and number of parses
    else:
        output_file.write((" ").join(sentence[0])+'\t'+str(0)+'\n') #writing file sentences and parses = 0
output_file.close()
      
time_n = datetime.datetime.now()      
print("The total time taken was:", (time_n-time_parser).total_seconds(), "seconds") #measure total time taken to parse all sentences and print parse trees

The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in the language of CFG


The given sentence is not in th

In [78]:
sentences =open("trees_number.txt", 'r') #load the file with test sentences and number of parses
filtered =[]
for sentence in sentences: 
    if int(sentence.split('\t')[-1]) <6 and int(sentence.split('\t')[-1]) >1: #filter sentences with 2-5 parses
        filtered.append((sentence.split('\t')[0].split(), int(sentence.split('\t')[-1])))


In [89]:
def saving_tree(sentence, i):
    """
    This function saves the filtered parse trees with two to five parses
    Parameters:
    sentence
    i(int): position of sentence
    """
    backpointer, chart = cky_recognizer(sentence[0])
    if backpointer and chart:
        parses = tree_parser(parent_node=start_symbol,i= 0,j=len(sentence[0]),backpointer=backpointer)
        for j,parse in enumerate(parses):
            cf = CanvasFrame()
            t = Tree.fromstring(str(parse).replace(',',''))
            t.pretty_print()
            #t.draw()
            tc = TreeWidget(cf.canvas(),t)
            cf.add_widget(tc,10,10)
            cf.print_to_file('images/'+str(i) + '-'+str(j)+'.ps')
            cf.destroy()

In [90]:
for I,each in enumerate(filtered):
    print(each)
    print(I)
    saving_tree(each,I) #saving image of tree

(['prices', '.'], 2)
0
         SIGMA            
    _______|________       
NOUN_NNS       pt_char_per
   |                |      
'prices'           '.'    

         SIGMA            
    _______|________       
VERB_VBZ       pt_char_per
   |                |      
'prices'           '.'    

(['show', 'availability', '.'], 3)
1
            SIGMA                     
    __________|_________               
   |                   JKA            
   |           _________|_______       
VERB_VB     NP_NN          pt_char_per
   |          |                 |      
 'show' 'availability'         '.'    

            SIGMA                     
    __________|_________               
   |                   JWB            
   |           _________|_______       
NOUN_NN    AVPNP_NN        pt_char_per
   |          |                 |      
 'show' 'availability'         '.'    

           SIGMA                     
   __________|_________               
  |                   JXI        

                   SIGMA                               
    _________________|______                            
   |                       JKB                         
   |                  ______|____________________       
   |               NP_NNS                        |     
   |         ________|______                     |      
   |        |             PP_NP                  |     
   |        |         ______|________            |      
VERB_VB  NOUN_NNS PREP_IN         NOUN_NP   pt_char_per
   |        |        |               |           |      
 'list' 'flights'  'from'       'cleveland'     '.'    

                  SIGMA                               
   _________________|______                            
  |                       GRY                         
  |         _______________|________                   
  |        |                       GRZ                
  |        |                ________|___________       
  |        |             PP_NP               

                                 SIGMA                                                          
           ________________________|______________                                               
          |                                      GMA                                            
          |                  _____________________|_____________                                 
          |                 |                                  GMB                              
          |                 |                             ______|_________________________       
          |                 |                          NP_NP                              |     
          |                 |              ______________|_____________                   |      
        NP_NNS              |          NOUN_NP                       PP_NP                |     
    ______|________         |       ______|_______               ______|_______           |      
ADJ_WPS         NOUN_NNS 

                                         SIGMA                                                            
   ________________________________________|____________                                                   
  |                                                    GDS                                                
  |        _____________________________________________|________                                          
  |       |                                                     GDT                                       
  |       |                                              ________|__________________________________       
  |       |                                           NP_NP                                         |     
  |       |                          ___________________|_________________                          |      
  |       |                       NP_NN                                   |                         |     
  |       |        ______________

         SIGMA                                                                                                  
    _______|_______                                                                                              
   |              DLX                                                                                           
   |        _______|_______________                                                                              
   |       |                      DLY                                                                           
   |       |        _______________|________________                                                             
   |       |       |                               DLZ                                                          
   |       |       |                ________________|_____________________________________________________       
   |       |       |             PP_NN                                                      

         SIGMA                                                                                                                                 
    _______|_______                                                                                                                             
   |              BJG                                                                                                                          
   |        _______|______                                                                                                                      
   |       |             BJH                                                                                                                   
   |       |        ______|_____________________                                                                                                
   |       |       |                           BJI                                                                                   